In [2]:
# python imports
import pandas as pd
import altair as alt
import requests
import numpy as np

# our imports
import intervals
from intervals import importScore 
import intervals.visualizations as viz
# import visualizations_demo as viz_demo

In [3]:
data_observations = requests.get('http://crimproject.org/data/observations/').json()
df_observations = pd.json_normalize(data_observations)


In [10]:
df_observations.iloc[0, :]

url                                       https://crimproject.org/data/observations/1/
id                                                                                   1
ema                                  1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...
musical_type                                                                      fuga
remarks                                                           Makes up a longer ID
created                                                      2017-06-19T19:22:54-04:00
updated                                               2022-02-03T18:01:55.831239-05:00
curated                                                                           True
observer.url                         https://crimproject.org/data/people/CRIM_Perso...
observer.name                                                               Ian Lorenz
observer.id                                                                        349
piece.url                            https:

## D.  CRIM Metadata as Networks 

* uses `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.
- The `segments` column in the interactive df shows the range of the measures the observation was found, for example: `['5-9', '14-18']` means that the observation have been found from measure 5 to measure 9, and again from measure 14 to measure 18.



### D.1  Network of PEns by Melodic Intervals between Entries

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_int` (`mt` = musical type, `pe` = periodic entries, `int`= intervals) and we would choose `interval_type` to be `melodic intervals`. The table's corresponding ema column is `ema`.

In [5]:
pen_networks, pen_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'musical_type', 'details.entry intervals', 'ema')

Exception: Please put either 'time' or 'melodic' for `type_interval`

**See all the available starting intervals**:

- the 'all' key means that this networks contains all of the pens.
- If you select a network with specific starting intervals (e.g: 8+), it only contains pens that start with 8+.

In [16]:
pen_networks

{'all': <class 'pyvis.network.Network'> |N|=2 |E|=340,
 'fuga': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'periodic entry': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'soggetto': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'cantus firmus': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'non-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 'imitative duo': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'homorhythm': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'counter soggetto': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'contrapuntal duo': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'cadence': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'non imitative duo': <class 'pyvis.network.Network'> |N|=0 |E|=0}

In [17]:
# show all the patterns mapped onto one network
pen_networks['all'].show('all_pen.html')


In [18]:
# show only one family of patterns mapped onto one network
pen_networks['8+'].show('networks/8+_pen.html')

KeyError: '8+'

### D.2  Network of Fugas by Melodic Intervals between Entries

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= musical type, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [ ]:
fuga_networks, fuga_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

In [ ]:
# we are interested in the patterns that start with 4-
# fuga_networks['4-'].show('networks/4-_fuga.html')
fuga_networks['4-'].show('4-_fuga.html')

In [ ]:
fuga_networks

### D.3. CRIM Relationships as Network of Pieces

* `plot_relationship_network` method

This method accepts a dataframe of CRIM relationships and generate a network of observations.

In the network:

- the nodes are the observations (labeled with the piece and measures they are in)
- the edges are the relationships. 
    - points from a model_observation to a derivative observation.
    - are weighted differently based on the type of the relationships.
    - are labeled with the relationship type
    - are colored based on the users' choice.

**WARNING**: Edge color isn't always a correct indicator of the relationship type.

The color of the edges is derived from the nodes; therefore, they would not be colored correctly when:

1. color='derivative'

model1 --(quotation)--> model2

model3 --(new material)--> model2

because of the first relationship, model2 and the two edges pointing to it would be colored with quotation's color.

2. color='model'

model1 --(quotation)--> model2

model1 --(new material)--> model3

because of the first relationship, model1 and the two out edges from model1 would be colored with quotation's color

In this case, we should pay attention to the edges' thickness (different weights mean different relationship types) or hover over the edges to see the relationship type labels.

**Seeing the big picture with the color modes being `model` or `derivative`**

We have two options for the `color` parameter. 

- `derivative` (default): the edges and nodes of derivative observations will be colored according to the relationships. 
- `model` (default): the edges and nodes of model observations will be colored according to the relationships. 

**Here I use df_relationships.head(300) instead of df_relationships because plotting all of the relationships usually takes really long**.

#### color=`derivative`

In [31]:
nt = viz.plot_relationship_network(df_relationships.head(300), color='derivative')
nt.show('small_relationships_network_derivative.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

#### color=`model`

In [30]:
nt2 = viz.plot_relationship_network(df_relationships.head(300), color='model')
nt2.show('small_relationships_network_model.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

### D.4  Networks with Specific Pieces (In Progress)

The two plots are too cluttered so we can't see the relationship_type between specific measures inside pieces. 

Selecting **specific models** with `selected_model_ids` and **specific derivatives** with`selected_derivative_ids` 

The first example shows all the movements of Févin's Missa Ave Maria as a network.  This reveals which parts of the model have been used where, since 'centers' on the map represent particular measure ranges from the motet.



In [32]:
# nt3 = viz_demo.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
#        'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
#        'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
# nt3.show('selected_derivatives.html')

nt3 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=[
       'CRIM_Mass_0005_3', 'CRIM_Mass_0005_2', 'CRIM_Mass_0005_1',
       'CRIM_Mass_0005_4', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('selected_derivatives.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

**Network for One Model**

* Here is the same network mapped from the perspective of a model

In [33]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008'], color='derivative')
nt4.show('selected_model.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'

#### Plotting pieces that are relevant to a list of interested pieces with `selected_members`
Having some of interest pieces, we want to plot all pieces that happen to connect to these pieces somehow.

In [34]:
nt5 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt5.show('selected_model_families.html')

AttributeError: module 'intervals.visualizations' has no attribute 'plot_relationship_network'